In [ ]:
import numpy as np
import importlib, os, datetime, pickle
from sus.protocol_designer import System, Protocol, Potential, Compound_Protocol
from sus.protocol_designer.protocol import sequential_protocol
from IPython import display
from IPython.display import HTML, Image
import matplotlib.animation as animation
from matplotlib.animation import FuncAnimation, PillowWriter
from quick_sim import setup_sim


# import edward_tools.fq_runner as fq_runner
from edward_tools.coupled_fq_potential import coupled_flux_qubit_non_linear_approx_pot, coupled_flux_qubit_non_linear_approx_force, \
coupled_flux_qubit_non_linear_approx_pot_special, coupled_flux_qubit_non_linear_approx_force_special, coupled_flux_qubit_non_linear_approx_pot_break_down
from edward_tools.visualization import animate_sim_flux_qubit, plotFidelityBarChart, separate_by_state_2
from edward_tools.initial_state_sampling import extra_constraint_00_and_11_only
from NAND_PARAMETERS import *
import importlib

import kyle_tools as kt
import matplotlib.pyplot as plt


from edward_tools import coupled_fq_protocol_library, cfq_runner
from edward_tools import coupled_fq_protocol_library

import edward_tools.cfq_batch_sweep as cfq_batch_sweep
import edward_tools.Analysis_tool.general_analysis_tools as general_analysis_tool
# from edward_tools.Analysis_tool.general_analysis_tools import show_phi_dc_with_time
import edward_tools.Analysis_tool.minimum_value_of_potential as minimum_value_of_potential
from edward_tools.couple_flux_qubit_metrics import fidelityEvaluation
from edward_tools import visualization

coupled_fq_protocol_library = importlib.reload(coupled_fq_protocol_library)
create_system = coupled_fq_protocol_library.create_system
get_potential_shot_at_different_t = coupled_fq_protocol_library.get_potential_shot_at_different_t
get_potential_shot_at_different_t_1D = coupled_fq_protocol_library.get_potential_shot_at_different_t_1D
create_simple_protocol_parameter_dict = coupled_fq_protocol_library.create_simple_protocol_parameter_dict
coupled_fq_runner = importlib.reload(cfq_runner)
coupled_fq_protocol_library = importlib.reload(coupled_fq_protocol_library)
create_system = coupled_fq_protocol_library.create_system
get_potential_along_a_1D_cutline = coupled_fq_protocol_library.get_potential_along_a_1D_cutline
plotCutlines = coupled_fq_protocol_library.plotCutlines

# parameters

### circuit parameters

In [ ]:
circuit_params_from_Chris = {
    "L": 5e-12 * 1, "T":4.2, "L_factor": 1, 
    "beta": 2.3, "d_beta": 0,
    "phi_1x": 0.61,   "phi_2x": 0.10, "phi_1xdc": 0, "phi_2xdc": 1.79, "M_12": 0.06
}

experiment_circuit_params = {
    "L": 5e-12, "T": 4.2, "L_factor": 1,
    "beta": 10, "d_beta": 0,
    "phi_1x": 0.0,   "phi_2x": 0.0, "phi_1xdc": -2.73, "phi_2xdc": 0,
    "M_12": 0.
}


circuit_params_best_work_done = {
    "L": 5e-12 * 1, "T":4.2, "L_factor": 1, 
    "beta": 2.3, "d_beta": 0,
    "phi_1x": 0.59,   "phi_2x": 0.09, "phi_1xdc": 0, "phi_2xdc": 1.90, "M_12": 0.052
}




circuit_params_chosen = experiment_circuit_params
experiment_comment = "Experiment 3 (2024-10-22): Failed KE extraction protocol simulation for Kyle's animation"

phi_1_x_on_12, phi_2_x_on_12, phi_1_dcx_on_12, phi_2_dcx_on_12, phi_1_x_on_21, phi_2_x_on_21, phi_1_dcx_on_21, phi_2_dcx_on_21, M_12_on, four_well, CE_1, CE_8, lower_H_wells, flip_H, lower_V_wells, flip_V, KE_extraction_protocol_for_CE_1, KE_extraction_protocol_for_CE_8 = generate_protocols_from_circuit_params(circuit_params_chosen)

In [ ]:
has_velocity = True

PHI_0 = 2.067833848 * 1e-15
k_B = 1.38e-23
T = circuit_params_chosen["T"]
# T = 0.1
# T = 7
k_BT = k_B * T

C_factor = 1
L_factor = 1
R_factor = 1
# I_m_factor = 50
# I_m_factor = 15
I_m_factor = 0
time_scale = 1.0



I_p_1, I_p_2 = 5e-6 , 5e-6  # Amp
I_m_1, I_m_2 = 7e-9 * I_m_factor, 7e-9 * I_m_factor                           # Amp
R_1, R_2 = 1 * R_factor, 1 * R_factor                                         # ohm
C_1, C_2 = 500e-15 * C_factor, 500e-15 * C_factor                             # F

L_1, L_2 = 140e-12 * L_factor, 140e-12 * L_factor                             # H 
L_1, L_2 = 5e-12 * L_factor, 5e-12 * L_factor                             # H 
freq = 1/np.sqrt(C_1 * L_1)
characteristic_time = np.sqrt(C_1 * C_factor * L_1 * L_factor)



In [ ]:

m_c = C_1
m_1 = C_1
m_2 = C_2
x_c = PHI_0 / (2 * np.pi)
time_scale_factor = 1
t_c = np.sqrt(L_1 * C_1)
v_c = x_c / t_c


U0_1 = m_c * x_c**2 / t_c**2 / k_BT
U0_2 = m_2 * x_c**2 / t_c**2 / k_BT
kappa_1, kappa_2, kappa_3, kappa_4 = 1/U0_1, 1/U0_1, 1/U0_1, 1/U0_1

lambda_1 = 2 * np.sqrt(L_1 * C_1) / (C_1 * R_1)
theta_1  = 1
eta_1    = np.sqrt(np.sqrt(L_1 * C_1)/ (R_1 * C_1)) * np.sqrt(2 * kappa_1 / 1**2)

lambda_2 = 2 * np.sqrt(L_1 * C_1) / (C_2 * R_2)
theta_2  = 1 / (C_2/C_1)
eta_2    = np.sqrt(np.sqrt(L_1 * C_1)/ (R_1 * C_1)) * np.sqrt(2 * kappa_2 * (R_1 * C_1**2) / (R_2 * C_2**2))

lambda_3 = 2 * np.sqrt(L_1 * C_1) / (C_1 * R_1)
theta_3  = 4
eta_3    = np.sqrt(np.sqrt(L_1 * C_1)/ (R_1 * C_1)) * np.sqrt(8 * kappa_3)

lambda_4 = 2 * np.sqrt(L_1 * C_1) / (C_2 * R_2)
theta_4  = 4 / (C_2/C_1)
eta_4    = np.sqrt(np.sqrt(L_1 * C_1)/ (R_1 * C_1)) * np.sqrt(8 * kappa_4 * (R_1 * C_1**2) / (R_2 * C_2**2))

gamma = 10


beta_1 = 2 * np.pi * L_1 * I_p_1 / PHI_0; 
beta_2 = 2 * np.pi * L_2 * I_p_2 / PHI_0;

beta_1 = 7
beta_2 = 7

# beta_1 = 2
# beta_2 = 2



d_beta_1 = 2 * np.pi * L_1 * I_m_1 / PHI_0; 
d_beta_2 = 2 * np.pi * L_2 * I_m_2 / PHI_0;


_damping_factor = 1
_lambda = np.array([lambda_1, lambda_2, lambda_3, lambda_4])
_theta  = np.array([theta_1, theta_2, theta_3, theta_4])
_eta  =   np.array([eta_1, eta_2, eta_3, eta_4])

v_1 = np.random.normal(0, np.sqrt(k_BT/m_1)) / v_c
v_2 = np.random.normal(0, np.sqrt(k_BT/m_2)) / v_c
v_3 = np.random.normal(0, np.sqrt(k_BT/(m_1/4))) / v_c
v_4 = np.random.normal(0, np.sqrt(k_BT/(m_2/4))) / v_c

In [ ]:
print(f"L_1 = {L_1 * 1e12:.3g}pH, T = {T}K")
print(f"freq = {freq / 1e9:.3g}GHz")
print(characteristic_time)
print(1/U0_1*50)

In [ ]:
U0_1

In [ ]:
k_BT / (x_c**2/L_1) * 50

### parameter setting

In [ ]:
"""
# step 0: modify parameters
- All the parameters are stored in a separate file PARAMETER_INPUT
- You can override some of the parameters here.
"""
params = {}
params['N'] = 1000
params['dt'] = 1/100
params['lambda'] = 1
params['beta'] = 1
params['sim_params'] = [_lambda, _theta, _eta]
params['target_work'] = None
params['applyOffset'] = False
params['measureWorkWithOffset'] = True
params['monitor_work_dist_in_whole_process'] = True # To monitor the work process
params['comment'] = experiment_comment
params['capacitance'] = np.array([C_1, C_2, C_1/4, C_2/4])
params['mass'] = np.array([1, 1, 1/4, 1/4])
params['v_c'] = x_c/t_c
params['k_BT'] = k_BT
params['U0'] = U0_1
params['as_step'] = np.s_[::10] # the time step to skep for the all_state
params['percentage'] = 1 # For what percentage of the total sample do you want to keep in the output all_state


In [ ]:
"""
# step 2: Define initial condition and protocol
"""
manual_domain=[np.array([-10, -10]), np.array([10, 10])]



# µ = 0.06, φ2xdc = 1.79, φ1x = 0.59, and φ2x = 0.10.
    
initial_parameter_dict = {
        "U0_1": U0_1,     "U0_2": U0_2,     "gamma_1": gamma,  "gamma_2": gamma,
        "beta_1": beta_1,   "beta_2": beta_2,   "d_beta_1": d_beta_1 ,   "d_beta_2": d_beta_2,
        "phi_1_x": phi_1_x_off,  "phi_2_x": phi_2_x_off,  "phi_1_dcx": phi_1_dcx_off,  "phi_2_dcx": phi_1_dcx_off,
        "M_12": M_12_off, 'x_c': x_c
}


In [ ]:
"""
# step 1: Define potential
"""
coupled_fq_default_param = [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, x_c]
[phi_1_bound, phi_2_bound, phi_1dc_bound, phi_2dc_bound] = np.array([4, 4, 4, 4])/time_scale_factor
contour_range = [300, 2000]
    
coupled_fq_domain = [[-phi_1_bound, -phi_2_bound, -phi_1dc_bound, -phi_2dc_bound], \
                     [phi_1_bound, phi_2_bound, phi_1dc_bound, phi_2dc_bound]]

# coupled_fq_pot = Potential(coupled_flux_qubit_pot_with_offset_at_00_xy, coupled_flux_qubit_force, 14, 4,\
#                            default_params = coupled_fq_default_param,  relevant_domain = coupled_fq_domain)

coupled_fq_pot = Potential(coupled_flux_qubit_non_linear_approx_pot, coupled_flux_qubit_non_linear_approx_force, 14, 4,\
                           default_params = initial_parameter_dict,  relevant_domain = coupled_fq_domain)

# coupled_fq_pot = Potential(coupled_flux_qubit_non_linear_approx_pot_special, coupled_flux_qubit_non_linear_approx_force_special, 14, 4,\
                           # default_params = initial_parameter_dict,  relevant_domain = coupled_fq_domain)

### protocol setting

In [ ]:
zeroDissipation = False
# zeroDissipation = True

saveAllStates = True

params['sim_params'] = [_lambda, _theta, _eta]

if zeroDissipation:
    params['sim_params'] = [_lambda * 0, _theta, _eta * 0]

params['circuit_parameters'] = {
    "C_factor":C_factor, "L_factor": L_factor, "R_factor": R_factor, "I_m_factor": I_m_factor, "T": T, 
    "I_p_1": I_p_1, "I_p_2": I_p_2, "I_m_1": I_m_1, "I_m_2": I_m_2,
    "R_1": R_1, "R_2": R_2, "C_1": C_1, "C_2": C_2, "L_1": L_1, "L_2": L_2, 
    "characteristic_time": np.sqrt(C_1 * C_factor * L_1 * L_factor),
    "phi_1_x_on": phi_1_x_on_12, "phi_2_x_on": phi_2_x_on_12,
    "phi_1_dcx_on": phi_1_dcx_on_12, "phi_2_dcx_on": phi_2_dcx_on_12, "M_12_on": M_12_on,
    "gamma": gamma
}


initial_state_used_choice = ["four_well", "after_CE_8_completed"]



# bookmark
initial_parameter_dict["phi_1_dcx"] = circuit_params_chosen["phi_1xdc"]
initial_parameter_dict["phi_2_dcx"] = circuit_params_chosen["phi_2xdc"]
initial_parameter_dict["phi_1_x"]   = circuit_params_chosen["phi_1x"]
initial_parameter_dict["phi_2_x"]   = circuit_params_chosen["phi_2x"]
initial_parameter_dict["M_12"]      = circuit_params_chosen["M_12"]

initial_parameter_dict["phi_1_dcx"] = 0
initial_parameter_dict["phi_2_dcx"] = 0
initial_parameter_dict["phi_1_x"]   = 0
initial_parameter_dict["phi_2_x"]   = 0
initial_parameter_dict["M_12"]      = 0

In [ ]:
initial_parameter_dict

### modify the protocol

In [ ]:
CE_1 = {
        "phi_1_x": phi_1_x_on_12, "phi_2_x": phi_2_x_on_12, "M_12": M_12_on, \
        "phi_1_dcx": phi_1_dcx_off, "phi_2_dcx": phi_2_dcx_on_12, "name":"CE_1"
}

KE_extraction_protocol_for_CE_1 = {
        "phi_1_x": phi_1_x_on_12, "phi_2_x": 0.00, "M_12": 0.00, \
        "phi_1_dcx": phi_1_dcx_on_12, "phi_2_dcx": phi_2_dcx_on_12,\
        "name":"KE_extraction_protocol"
}

crazy_protocol = {
        "phi_1_x": 0.5, "phi_2_x": 0, "M_12": 0, \
        "phi_1_dcx": -2.65, "phi_2_dcx": 0, "name":"crazy_CE_1"
}

crazy_protocol_2 = {
        "phi_1_x": 0.0, "phi_2_x": 0, "M_12": 0, \
        "phi_1_dcx": -2.72, "phi_2_dcx": 0, "name":"crazy_CE_1"
}

crazy_protocol_3 = {
        "phi_1_x": 0, "phi_2_x": 0, "M_12": 0., \
        "phi_1_dcx": -2.75, "phi_2_dcx": -2.90, "name":"crazy_CE_1"
}

crazy_protocol_4 = {
        "phi_1_x": 0, "phi_2_x": 0, "M_12": 0., \
        "phi_1_dcx": -2.75, "phi_2_dcx": -2.75, "name":"crazy_CE_1"
}


crazy_protocol_5 = {
        "phi_1_x": 0, "phi_2_x": 0, "M_12": 0, \
        "phi_1_dcx": -np.pi, "phi_2_dcx": 0, "name":"crazy_CE_1"
}

In [ ]:
protocol_type = "CE"

if protocol_type == "CE":
    mapping_dict = mapping_state_1_to_state_2_dict_CE
if protocol_type == "NAND":
    mapping_dict = mapping_state_1_to_state_2_dict_NAND



protocol_list = [
    # forward
    create_CE_Protocol(100, crazy_protocol),
    create_CE_Protocol(500, crazy_protocol_2),
    # create_CE_Protocol(100, crazy_protocol_3),
    # create_CE_Protocol(100, crazy_protocol_4),
    # create_CE_Protocol(300, four_well)
]

# protocol_list = [
#     # create_CE_Protocol(100, crazy_protocol_3),
#     # create_CE_Protocol(100, crazy_protocol_4),
#     create_CE_Protocol(100, crazy_protocol_5),
#     create_CE_Protocol(100, crazy_protocol_5)
#     # create_CE_Protocol(100, four_well),
# ]

In [ ]:
protocol_list

# setting protocol (click here!)

# first initialization

### create initial state 

In [ ]:
%%capture
initial_state_array = ["saved_init_state/beta=10_T=4.2K_N=1000_initial_state.npy"]
initial_state_chosen = initial_state_array[0]


regenerate_init_state = False
# regenerate_init_state = True
if regenerate_init_state:
    initial_potential_protocol_list = [create_CE_Protocol(200*100*params['dt'], four_well)]
    init_params = params.copy()
    # friction_factor = 1
    # init_params['sim_params'] = [init_params['sim_params'][0] * friction_factor, init_params['sim_params'][1], init_params['sim_params'][2] * friction_factor]
    init_params['dt'] = 1/100
    init_state_saved = cfq_batch_sweep.create_initial_state(initial_parameter_dict, initial_potential_protocol_list, coupled_fq_pot, init_params)
else:
    init_state_saved = np.load(initial_state_chosen)

init_state_used = init_state_saved

# np.save("four_well_default_init_state_N_1000.npy", init_state_saved)

# particle_index = visualization.separate_by_state_2(init_state_saved[:, (0, 1), :])
# init_state_used = init_state_saved[particle_index["00"]]
# [1.3, 0.3, 0.3, 0.3, 1]

In [ ]:
plt.scatter(init_state_saved[..., 0, 0], init_state_saved[..., 1, 0])

### create cfqr object

In [ ]:
"""
# step 3: create the relevant storage protocol and computation protocol
"""
def modifiedFunction(t, result):
    return result


computation_protocol_parameter_dict = coupled_fq_protocol_library.customizedProtocol(initial_parameter_dict, \
                                                                    protocol_list)
storage_protocol, comp_protocol = create_system(computation_protocol_parameter_dict, modifiedFunction = None)

cfqr = cfq_runner.coupledFluxQubitRunner(potential = coupled_fq_pot, params = params, \
                                                storage_protocol= storage_protocol, \
                                                computation_protocol= comp_protocol, \
                                         protocol_list = protocol_list, \
                                        has_velocity=has_velocity)

protocol_time_array, protocol_time_index_array, protocol_all_time_array, protocol_time_all_index_array = cfqr.createProtocolTimeArray(protocol_list, params)
cfqr.initialize_sim()
# cfqr.set_sim_attributes(init_state=init_state_saved)
cfqr.set_sim_attributes(init_state=init_state_used)
print(f"mean KE = {np.mean(cfqr.system.get_kinetic_energy(init_state_used))}")
print(f"characteristic_time = {characteristic_time * 10e9:.3g}ns")
print(f"total time = {protocol_time_array[-1] * characteristic_time * 10e9:.3g}ns")

In [ ]:
# before check opo

In [ ]:
def plot_3_well_potential(phi_1x, mu):
    phi_1dc_array = np.linspace(-3.0, -2.7, 10)
    phi_1_array = np.linspace(-4, 4, 1000)
    phi_2 = 3.0
    phi_2dc = 0

    params_list_used = cfqr.protocol.get_params(0)
    phi_1xdc_array = list(np.linspace(-2.4, -2.72, 5))
    mu_array = np.linspace(0.05, 0.10, 10)
    phi_1x_array  = np.linspace(0., 0.1, 10)
    phi_2x_array  = np.linspace(0, 10, 10)
    for x in phi_1xdc_array:
        fig = plt.figure(figsize= [20, 5])
        ax_0 = fig.add_subplot(1, 4, 1, projection='3d')
        ax_1 = fig.add_subplot(1, 4, 2)
        ax_2 = fig.add_subplot(1, 4, 3, projection='3d')
        ax_3 = fig.add_subplot(1, 4, 4)

        params_list_used[8] = 0.0 # phi_1x
        params_list_used[9] = 0 # phi_2x"
        params_list_used[10] = x # phi_1xdc
        params_list_used[11] = 0 # phi_2xdc
        params_list_used[12] = 0.1 # mu
        params_at_t_str = [f"{key}: {value:.3g}" for key, value in zip(cfqr.protocol_key[-6:-1], params_list_used[-6:-1])]
        fig.suptitle(f"phi_1xdc = {params_list_used[10]:.3g}, phi_1x = {params_list_used[8]:.3g}, phi_2x = {params_list_used[9]:.3g}, mu = {params_list_used[12]:.3g}") 

        potential_min = []
        for _phi_1dc in phi_1dc_array:
            for _phi_2 in [3.0]:
                potential = cfqr.potential.potential(phi_1_array, _phi_2, _phi_1dc, phi_2dc, params_list_used)
                ax_0.plot(phi_1_array, np.full(len(phi_1_array), fill_value=_phi_1dc), potential, label = f"{_phi_1dc:.3g}")
                ax_1.plot(phi_1_array, potential, label = f"phi_1dc = {_phi_1dc:.3g}")

                potential = cfqr.potential.potential(phi_1_array, -_phi_2, _phi_1dc, phi_2dc, params_list_used)
                ax_2.plot(phi_1_array, np.full(len(phi_1_array), fill_value=_phi_1dc), potential, label = f"{_phi_1dc:.3g}")
                ax_3.plot(phi_1_array, potential, label = f"phi_1dc = {_phi_1dc:.3g}")

            
                ax_0.set_xlim(-4, 4)
    
                min_y, max_y = -1000, 800
                # ax_0.set_zlim(min_y, max_y)
                ax_1.set_ylim(min_y, max_y)
                ax_3.set_ylim(min_y, max_y)
        plt.show()

In [ ]:
plot_3_well_potential(phi_1x = 0.01, mu = -0.01)

# actual simulations

In [ ]:
## importlib.reload(cfq_batch_sweep)

# 0.5 * (x - 0.59)^2 + 0.5 * (y - 0.1)^2 + 2.3 * cos(x) + 2.3 * cos(y)
# step 5: perform simulations


manual_domain=[np.array([-5, -5]), np.array([5, 5])]

simResult = cfq_batch_sweep.simulateSingleCoupledFluxQubit(params, initial_parameter_dict, protocol_list, \
                        potential = coupled_fq_pot, potential_default_param = coupled_fq_default_param, mapping_state_1_to_state_2_dict = mapping_dict,         
                        initial_state = init_state_used, manual_domain = manual_domain, \
                        phi_1_dcx = phi_1_dcx,  phi_2_dcx = phi_2_dcx, measure_all_states=saveAllStates, \
                        has_velocity = has_velocity)


In [ ]:
print(f"characteristic_time = {characteristic_time * 10e9:.3g}ns, freq = {1/characteristic_time * 10e-9:.3g} GHz")

In [ ]:
particle_index = cfqr.getIndexOfParticles()

In [ ]:
particleInformation = {
    "showParticles": True,
    "project_item": ["00", "01", "10", "11"],
    "particle_opacity": 0.5,
    "pColor": {"00": "#061DF7", "01": "red", "10": "#3FC7F2", "11": "#F187F4"}
}

In [ ]:
# init_state = simResult['cfqr'].get_all_state()[:,0,...]
# final_state = simResult['cfqr'].get_all_state()[:,-1,...]


# fig, ax = plt.subplots(2, 6, figsize = [30, 10])
# ax_flatten = ax.flatten()
# for key, index in particle_index.items():
#     ax_flatten[0].scatter(init_state[index,0,0], init_state[index,1,0], color = particleInformation["pColor"][key])
#     ax_flatten[1].scatter(final_state[index,0,0], final_state[index,1,0], color = particleInformation["pColor"][key])
# ax_flatten[0].set_title("initial_state: phi_1 vs phi_2")
# ax_flatten[1].set_title("final_state: phi_1 vs phi_2")


# for key, index in particle_index.items():
#     ax_flatten[2].scatter(init_state[index,2,0], init_state[index,3,0], color = particleInformation["pColor"][key])
#     ax_flatten[3].scatter(final_state[index,2,0], final_state[index,3,0], color = particleInformation["pColor"][key])
# ax_flatten[2].set_title("initial_state: phi_1dc vs phi_2dc")
# ax_flatten[3].set_title("final_state: phi_1dc vs phi_2dc")



# for key, index in particle_index.items():
#     ax_flatten[4].scatter(init_state[index,0,0], init_state[index,2,0], color = particleInformation["pColor"][key])
#     ax_flatten[5].scatter(final_state[index,0,0], final_state[index,2,0], color = particleInformation["pColor"][key])
# ax_flatten[4].set_title("initial_state: phi_1 vs phi_1dc")
# ax_flatten[5].set_title("final_state: phi_1 vs phi_1dc")

# for key, index in particle_index.items():
#     ax_flatten[6].scatter(init_state[index,0,0], init_state[index,3,0], color = particleInformation["pColor"][key])
#     ax_flatten[7].scatter(final_state[index,0,0], final_state[index,3,0], color = particleInformation["pColor"][key])
# ax_flatten[6].set_title("initial_state: phi_1 vs phi_2dc")
# ax_flatten[7].set_title("final_state: phi_1 vs phi_2dc")


# for key, index in particle_index.items():
#     ax_flatten[8].scatter(init_state[index,1,0], init_state[index,2,0], color = particleInformation["pColor"][key])
#     ax_flatten[9].scatter(final_state[index,1,0], final_state[index,2,0], color = particleInformation["pColor"][key])
# ax_flatten[8].set_title("initial_state: phi_2 vs phi_1dc")
# ax_flatten[9].set_title("final_state: phi_2 vs phi_1dc")

# for key, index in particle_index.items():
#     ax_flatten[10].scatter(init_state[index,1,0], init_state[index,3,0], color = particleInformation["pColor"][key])
#     ax_flatten[11].scatter(final_state[index,1,0], final_state[index,3,0], color = particleInformation["pColor"][key])
# ax_flatten[10].set_title("initial_state: phi_2 vs phi_2dc")
# ax_flatten[11].set_title("final_state: phi_2 vs phi_2dc")


# for _ax in ax_flatten:
#     _ax.set_xlim(-4, 4)
#     _ax.set_ylim(-4, 4)
# plt.show()

## to plot motion of particles in all the planes

In [ ]:
all_states = simResult['cfqr'].get_all_state()
phi_1 = all_states[..., 0, 0]
phi_2 = all_states[..., 1, 0]
phi_1dc = all_states[..., 2, 0]
phi_2dc = all_states[..., 3, 0]


In [ ]:
frame_skip = 20
targetTimeArray = protocol_all_time_array[simResult['cfqr'].sim.target_step_index]
animation_time_array = targetTimeArray[::frame_skip]
animation_states = all_states[:, ::frame_skip, ...]

fig, ax = plt.subplots(2, 5, figsize=[20, 7])
plot_label_1 = [("phi_1", "phi_2"), ("phi_1dc", "phi_2dc"), ("phi_1", "phi_1dc"), ("phi_1", "phi_2dc"), ("phi_2", "phi_1dc"), ("phi_2", "phi_2dc")]
plot_label_2 = ["phi_1", "phi_2", "phi_1dc", "phi_2dc"]

ax_flatten = ax.flatten()
ax_flatten_1 = [ax_flatten[i] for i in [0, 1, 2, 5, 6, 7]]
ax_flatten_2 = [ax_flatten[i] for i in [3, 4, 8, 9]]
phi_1_array = np.linspace(-4, 4)
phi_1dc_array = np.linspace(-4, 4)

for _ax, (xlabel, ylabel) in zip(ax_flatten_1, plot_label_1):
    _ax.set_xlim(-4, 4)
    _ax.set_ylim(-4, 4)
    _ax.set_xlabel(xlabel)
    _ax.set_ylabel(ylabel)    
    _ax.set_aspect(1)

for _ax, xlabel in zip(ax_flatten_2, plot_label_2):
    _ax.set_xlim(-3.5, 3.5)
    _ax.set_ylim(-4500, 1000)
    _ax.set_xlabel(xlabel)
    _ax.set_ylabel("PE")    
    # _ax.set_aspect(1)


plot_axis_1 = [[0, 1], [2, 3], [0, 2], [0, 3], [1, 2], [1, 3]]
plot_axis_2 = [0, 1, 2, 3]

scatter_array_1 = []
scatter_array_2 = []
potential_min_plot_array = {}
potential_max_plot_array = {}

potential_min_1dc_plot_array = {}
potential_max_1dc_plot_array = {}

for key, index in particle_index.items():
    for _i, (_x, _y) in enumerate(plot_axis_1):
        _scatter = ax_flatten_1[_i].scatter(all_states[index, 0, _x, 0], all_states[index, 0, _y, 0], c = particleInformation['pColor'][key])
        scatter_array_1.append(_scatter)

    for _i in plot_axis_2:
        potential = simResult['cfqr'].system.get_potential(animation_states[index, 0, ...], 0)
        _scatter = ax_flatten_2[_i].scatter(all_states[index, 0, _i, 0], potential, c = particleInformation['pColor'][key])
        scatter_array_2.append(_scatter)
        
        params_at_0 = simResult['cfqr'].protocol.get_params(0)
        phi_1_min_array = np.min(animation_states[index, 0, 0, 0], axis = 0)
        phi_1_max_array = np.max(animation_states[index, 0, 0, 0], axis = 0)
        phi_2_array = np.mean(animation_states[index, 0, 1, 0], axis = 0)
        phi_1dc_min_array = np.min(animation_states[index, 0, 2, 0], axis = 0)
        phi_1dc_max_array = np.max(animation_states[index, 0, 2, 0], axis = 0)
        phi_2dc_array = np.mean(animation_states[index, 0, 3, 0], axis = 0)
            
        
        if _i == 0:
            
            potential_min_array = coupled_flux_qubit_non_linear_approx_pot(phi_1_array, phi_2_array, phi_1dc_min_array, phi_2dc_array, params_at_0)
            _potetnial_min_plot = ax_flatten_2[_i].plot(phi_1_array, potential_min_array, c = particleInformation['pColor'][key], linestyle= "--")[0]
            potential_min_plot_array[key] = _potetnial_min_plot
        
            potential_max_array = coupled_flux_qubit_non_linear_approx_pot(phi_1_array, phi_2_array, phi_1dc_max_array, phi_2dc_array, params_at_0)
            _potetnial_max_plot = ax_flatten_2[_i].plot(phi_1_array, potential_max_array, c = particleInformation['pColor'][key])[0]
            potential_max_plot_array[key] = _potetnial_max_plot
            
        if _i == 2:
            potential_min_array = coupled_flux_qubit_non_linear_approx_pot(phi_1_min_array, phi_2_array, phi_1dc_array, phi_2dc_array, params_at_0)
            _potetnial_min_plot = ax_flatten_2[_i].plot(phi_1dc_array, potential_min_array, c = particleInformation['pColor'][key], linestyle= "--")[0]
            potential_min_1dc_plot_array[key] = _potetnial_min_plot
        
            potential_max_array = coupled_flux_qubit_non_linear_approx_pot(phi_1_max_array, phi_2_array, phi_1dc_array, phi_2dc_array, params_at_0)
            _potetnial_max_plot = ax_flatten_2[_i].plot(phi_1dc_array, potential_max_array, c = particleInformation['pColor'][key])[0]
            potential_max_1dc_plot_array[key] = _potetnial_max_plot
        
        
def animate(i):
    counter_1, counter_2 = 0, 0
    _t = animation_time_array[i]
    simResult['cfqr'].protocol.get_params(animation_time_array[i])
    params_at_t_str = [f"{key}: {value:.3g}" for key, value in zip(simResult['cfqr'].protocol_key[-6:-1], simResult['cfqr'].protocol.get_params(animation_time_array[i])[-6:-1])]
    fig.suptitle(f"t = {animation_time_array[i]} + {params_at_t_str}")
    
    for key, index in particle_index.items():
        
        for _i, (_x, _y) in enumerate(plot_axis_1):
            scatter_array_1[counter_1].set_offsets(np.c_[animation_states[index, i, _x, 0], animation_states[index, i, _y, 0]])
            counter_1 += 1

        for _i in plot_axis_2:
            potential_i = simResult['cfqr'].system.get_potential(animation_states[index, i, ...], animation_time_array[i])
            scatter_array_2[counter_2].set_offsets(np.c_[animation_states[index, i, _i, 0], potential_i])

            params_at_t = simResult['cfqr'].protocol.get_params(_t)
            phi_1_min_array = np.min(animation_states[index, i, 0, 0], axis = 0)
            phi_1_max_array = np.max(animation_states[index, i, 0, 0], axis = 0)
            
            phi_2_array = np.mean(animation_states[index, i, 1, 0], axis = 0)

            phi_1dc_min_array = np.min(animation_states[index, i, 2, 0], axis = 0)
            phi_1dc_max_array = np.max(animation_states[index, i, 2, 0], axis = 0)
            
            phi_2dc_array = np.mean(animation_states[index, i, 3, 0], axis = 0)
            
            if _i == 0:    
                potential_min_array = coupled_flux_qubit_non_linear_approx_pot(phi_1_array, phi_2_array, phi_1dc_min_array, phi_2dc_array, params_at_t)
                potential_min_plot_array[key].set_xdata(phi_1_array)
                potential_min_plot_array[key].set_ydata(potential_min_array)
                
                potential_max_array = coupled_flux_qubit_non_linear_approx_pot(phi_1_array, phi_2_array, phi_1dc_max_array, phi_2dc_array, params_at_t)
                potential_max_plot_array[key].set_xdata(phi_1_array)
                potential_max_plot_array[key].set_ydata(potential_max_array)
            
            if _i == 2:
                potential_min_array = coupled_flux_qubit_non_linear_approx_pot(phi_1_min_array, phi_2_array, phi_1dc_array, phi_2dc_array, params_at_t)
                potential_min_1dc_plot_array[key].set_xdata(phi_1dc_array)
                potential_min_1dc_plot_array[key].set_ydata(potential_min_array)
                
                potential_max_array = coupled_flux_qubit_non_linear_approx_pot(phi_1_max_array, phi_2_array, phi_1dc_array, phi_2dc_array, params_at_t)
                potential_max_1dc_plot_array[key].set_xdata(phi_1dc_array)
                potential_max_1dc_plot_array[key].set_ydata(potential_max_array)
            
            counter_2 += 1

ani = FuncAnimation(fig, animate, interval=100, frames=len(animation_time_array))
writer = animation.PillowWriter(fps=30, metadata=dict(artist='Me'), bitrate=1800)
ani.save('scatter.gif', writer=writer)
plt.close()

In [ ]:
Image(filename="scatter.gif")

In [ ]:
protocol_list

In [ ]:
animation_time_array[::10]

In [ ]:
# import numpy as np
# from mpl_toolkits.mplot3d import axes3d
# import matplotlib.pyplot as plt

# fig = plt.figure()
# ax = fig.add_subplot(111, projection="3d")
# phi_1_array = np.linspace(-5, 5, 100)
# phi_1dc_array = np.linspace(-5, 0, 100)
# phi_2 = 3
# phi_2dc = 0
# Y, X = np.meshgrid(phi_1_array, phi_1dc_array)
# Z = simResult['cfqr'].potential.potential(Y, phi_2, X, phi_2dc, params_at_t)
# ax.plot_surface(X, Y, Z, cmap="autumn_r", lw=0.5, rstride=1, cstride=1, alpha=0.5)
# ax.contour(X, Y, Z, 10,  cmap="autumn_r", linestyles="solid", offset=-1)
# ax.contour(X, Y, Z, 10, colors="k", linestyles="solid")
# plt.show()


In [ ]:
frame_skip = 250
targetTimeArray = protocol_all_time_array[simResult['cfqr'].sim.target_step_index]
animation_time_array = targetTimeArray[::frame_skip]
animation_states = all_states[:, ::frame_skip, ...]

index = particle_index['01']
phi_1dc_array = np.linspace(-3, -2.5, 20)
phi_1_array = np.linspace(-4, 4, 1000)
phi_2 = 3
phi_2dc = 0
# phi_1dc_array = np.linspace(-4, 4, 100)

params_list_used = simResult['cfqr'].protocol.get_params(0)
for x in [-2.0, -2.5, -2.6, -2.7]:
    fig = plt.figure(figsize= [10, 5])
    ax_0 = fig.add_subplot(1, 2, 1, projection='3d')
    ax_1 = fig.add_subplot(1, 2, 2)

    params_list_used[8] = -0.02 # phi_1x
    params_list_used[9] = 0.0 # phi_2x"
    params_list_used[10] = x # phi_1xdc
    params_list_used[11] = 0 # phi_2xdc
    params_list_used[12] = 0.08 # mu
    params_at_t_str = [f"{key}: {value:.3g}" for key, value in zip(simResult['cfqr'].protocol_key[-6:-1], params_list_used[-6:-1])]
    fig.suptitle(f"phi_1xdc = {x}") 
    
    potential_min = []
    for _phi_1dc in phi_1dc_array:
        potential = simResult['cfqr'].potential.potential(phi_1_array, phi_2, _phi_1dc, phi_2dc, params_list_used)
        ax_0.plot(phi_1_array, np.full(len(phi_1_array), fill_value=_phi_1dc), potential, label = f"{_phi_1dc:.3g}")
        ax_1.plot(phi_1_array, potential, label = f"phi_1dc = {_phi_1dc:.3g}")
        potential_min.append(min(potential))
    ax_0.set_xlim(-4, 4)
    # ax_0.set_zlim(min(potential_min) - 100, min(potential_min)+1000)
    # ax_1.set_ylim(-1400, -1000)
    # ax_1.legend()
    plt.show()

In [ ]:
simResult['cfqr'].protocol_key

In [ ]:
potential.shape

In [ ]:
import matplotlib.pylab as pl
from mpl_toolkits.mplot3d import Axes3D
import numpy as np

x   = np.linspace(1,5,100)
y1  = np.ones(x.size)
y2  = np.ones(x.size)*2
y3  = np.ones(x.size)*3
z   = np.sin(x/2)

pl.figure()
ax = pl.subplot(projection='3d')
ax.plot(x, y1, z, color='r')
ax.plot(x, y2, z, color='g')
ax.plot(x, y3, z, color='b')
ax.set_xlabel('Day')
ax.set_zlabel('Resistance (%)')

In [ ]:
frame_skip = 250
targetTimeArray = protocol_all_time_array[simResult['cfqr'].sim.target_step_index]
animation_time_array = targetTimeArray[::frame_skip]
animation_states = all_states[:, ::frame_skip, ...]

# fig, ax = plt.subplots(1, 2, figsize = [10, 5])

# ax[0].set_xlim(-4, 4)
# ax[0].set_xlabel("phi_1")
# ax[0].set_ylabel("PE")    

scatter_array_1 = []
scatter_array_2 = []
potential_plot_array = []

index = particle_index['01']
phi_1dc_array = np.linspace(-3, 0, 20)
# phi_1_array = np.linspace(-4, 4, 100)
# phi_1dc_array = np.linspace(-4, 4, 100)


for _i, _t in enumerate(animation_time_array):
    params_at_t = simResult['cfqr'].protocol.get_params(_t)
    params_at_t_str = [f"{key}: {value:.3g}" for key, value in zip(simResult['cfqr'].protocol_key[-6:-1], simResult['cfqr'].protocol.get_params(_t)[-6:-1])]
    plt.title(f"t = {_t} + {params_at_t_str}")    
    
    for _phi_1dc in phi_1dc_array[0:5]:
        _phi_2 = np.mean(animation_states[index, _i, 1, 0], axis = 0)
        _phi_2dc = np.mean(animation_states[index, _i, 3, 0], axis = 0)
        potential = simResult['cfqr'].potential.potential(phi_1_array, _phi_2, _phi_1dc, _phi_2dc, params_at_t)
        # potential_plot_array[key].plot(phi_1_array, potential_array, c = particleInformation['pColor'][key])
        plt.plot(phi_1_array, potential, label = f"{_phi_1dc:.3g}")
    plt.legend()
    plt.show()
# def animate(i):
#     _t = animation_time_array[i]
#     simResult['cfqr'].protocol.get_params(animation_time_array[i])
#     params_at_t_str = [f"{key}: {value:.3g}" for key, value in zip(simResult['cfqr'].protocol_key[-6:-1], simResult['cfqr'].protocol.get_params(animation_time_array[i])[-6:-1])]
#     plt.title(f"t = {animation_time_array[i]} + {params_at_t_str}")    
    
#     for _phi_1dc, _plot in zip(phi_1dc_array, potential_plot_array):
#         params_at_t = simResult['cfqr'].protocol.get_params(_t)
#         _phi_1dc = np.mean(animation_states[index, i, 1, 0], axis = 0)
#         _phi_2dc = np.mean(animation_states[index, i, 3, 0], axis = 0)
#         potential = simResult['cfqr'].potential.potential(phi_1_array, _phi_1dc, _phi_2, _phi_2dc, params_at_t)
#         # potential_plot_array[key].plot(phi_1_array, potential_array, c = particleInformation['pColor'][key])
#         _plot.set_xdata(phi_1_array)
#         _plot.set_ydata(potential)

        

# ani = FuncAnimation(fig, animate, interval=100, frames=len(animation_time_array))
# writer = animation.PillowWriter(fps=30, metadata=dict(artist='Me'), bitrate=1800)
# ani.save('scatter.gif', writer=writer)
# plt.close()

# sim Results

### fidelity

In [ ]:
import sys, importlib
# importlib.reload(sys.modules['edward_tools.jupyter_helper_function'])
from edward_tools.jupyter_helper_function import work_analysis, KE_analysis, single_particle_analysis, fidelity_analysis, work_done_of_each_type
fidelity_analysis(simResult, params, initial_parameter_dict, protocol_list)
print(simResult['simulation_data']['pickle_save_path'])

# work done of each type

In [ ]:
work_done_of_each_type(simResult)

# work analysis

In [ ]:
# work_analysis(simResult, params, protocol_list)

### animation

In [ ]:

importlib.reload(coupled_fq_protocol_library)

In [ ]:
def cutlineDataGeneratingFunction(initial_parameter_dict, direction = 'v', color = 'red', guess = [(-2, -2)]):
    coord_index = 0 if direction == "v" else 1
    def tracking_minimum_point(_t):
        return  [cfqr.get_minimum_at_t_for_a_set_of_parameter(_t, initial_parameter_dict, guess = guess)[0][coord_index], direction, color]
    return tracking_minimum_point

In [ ]:
pColor = {"00": "#061DF7", "01": "red", "10": "#3FC7F2", "11": "#F187F4"}

if L_factor == 1:
    graph_setting = {
        0.1: {"vmin": 1000, "vmax": 2000, "ymin": 1000, "ymax": 3000},
        0.5: {"vmin": 100, "vmax": 400, "ymin": 100, "ymax": 400},
        1: {"vmin": 100, "vmax": 400, "ymin": 100, "ymax": 400},
        2: {"vmin": 50, "vmax": 250, "ymin": 50, "ymax": 250},    
        4.2: {"vmin": 600, "vmax": 2000, "ymin": 600, "ymax": 3000},
        5: {"vmin": 0, "vmax": 100, "ymin": 0, "ymax": 150},
        7: {"vmin": 0, "vmax": 100, "ymin": 0, "ymax": 150},
        20: {"vmin": 0, "vmax": 20, "ymin": 0, "ymax": 20},
        100: {"vmin": 0, "vmax": 20, "ymin": 0, "ymax": 20},
        1000: {"vmin": 0, "vmax": 1, "ymin": 0, "ymax": 1},
    }

if L_factor == 1.5:
    graph_setting = {
        0.1: {"vmin": 300, "vmax": 2000, "ymin": 300, "ymax": 2000},
    }

    
if L_factor == 5:
    graph_setting = {
        0.1: {"vmin": -1500, "vmax": 800, "ymin": -1500, "ymax": 800},
    }

if L_1 <=6e-11:
     graph_setting = {
        4.2: {"vmin": 500, "vmax": 3000, "ymin": 500, "ymax": 3000},
    }

In [ ]:
import sys, importlib
importlib.reload(sys.modules['edward_tools.coupled_fq_protocol_library'])
# from edward_tools.coupled_fq_protocol_library import work_analysis, KE_analysis, single_particle_analysis

### without cutlineGeneratingFunction

In [ ]:
contourData = {
    "manual_domain": [np.array([-5, -5]), np.array([5, 5])],
    "contour_range": [-2000, 2000], "domain_limit": [-4, 4],
    "title": None
}

cutlineInformation = {
    "cutlineList": [(3, "h","red")],
    "cutlineGeneratingFunction": None,
    "cutlineXLimit": [-4,4],
    "cutlineYLimit": [-3000, 0]
}

particleInformation = {
    "showParticles": True,
    "project_item": ["11", "01"],
    "project_axis": "phi_1",
    "particle_opacity": 0.5,
    "pColor": {"00": "#061DF7", "01": "red", "10": "#3FC7F2", "11": "#F187F4"}
}



In [ ]:
import importlib

In [ ]:
importlib.reload(coupled_fq_protocol_library)

In [ ]:
simResult["cfqr"].getReducedTimeArray()

In [ ]:
all_state = simResult['cfqr'].sim.output.all_state['states']
animation_play = True
# animation_play = False
if animation_play:
    frame_skip = 50
    targetTimeArray = protocol_all_time_array[simResult['cfqr'].sim.target_step_index]
    animation_time_array = targetTimeArray[::frame_skip]
    animation_states = all_state[:, ::frame_skip, ...]
    print("geenerated at ", datetime.datetime.now())

    
    ani, fig, ax = coupled_fq_protocol_library.animate_sim_flux_qubit_with_cutline_and_projection(simResult['cfqr'], time_array=animation_time_array, 
                state_array = animation_states, plot_axis= [0,2], ax0_title = f"T = {T}K, L = {L_1 * 1e12}pH", params=params, cutlineGeneratingFunction = False, 
                cutlineInformation = cutlineInformation, particleInformation = particleInformation, contourData = contourData,
                pColor=pColor, contour_range = [0, 500], cutlineXLimit=[-4, 4], cutlineYLimit=[0, 500])
    fig.tight_layout()
    writer = animation.PillowWriter(fps=10, metadata=dict(artist='Me'), bitrate=1800)
    ani.save('scatter.gif', writer=writer)
    plt.close()
    


In [ ]:
Image(filename="scatter.gif")

In [ ]:
import sys, importlib
importlib.reload(sys.modules['edward_tools.coupled_fq_protocol_library'])
# from edward_tools.coupled_fq_protocol_library import work_analysis, KE_analysis, single_particle_analysis
animation_play = True
# animation_play = False
# contourData = {
#     "manual_domain": [np.array([-4, -4]), np.array([4, 4])],
#     "contour_range": [350, 1500], "domain_limit": [-3, 3],
#     "title": None
# }
contourData = {
    "manual_domain": [np.array([-4, -4]), np.array([4, 4])],
    "contour_range": [700, 1800], "domain_limit": [-3, 3],
    "title": None
}

cutlineInformation = {
    "cutlineList": [(2.3, "v","red")],
    "cutlineGeneratingFunction": None,
    "cutlineXLimit": [-4,4],
    "cutlineYLimit": [200, 1700]
}

particleInformation = {
    "showParticles": True,
    "project_item": ["10", "11"],
    "particle_opacity": 0.5,
    "pColor": {"00": "#061DF7", "01": "red", "10": "#3FC7F2", "11": "#F187F4"}
}


In [ ]:

animation_play = True
# animation_play = False
if animation_play:
    frame_skip = 20
    targetTimeArray = protocol_all_time_array[simResult['cfqr'].sim.target_step_index]
    animation_time_array = targetTimeArray[::frame_skip]
    animation_states = all_state[:, ::frame_skip, ...]
    print("geenerated at ", datetime.datetime.now())

    
    ani, fig, ax = coupled_fq_protocol_library.animate_sim_flux_qubit_with_cutline_and_projection(simResult['cfqr'], time_array=animation_time_array, state_array = animation_states, plot_axis= [0,1],
                                                                                   vmin=graph_setting[T]['vmin'], vmax=graph_setting[T]['vmax'], ax0_title = f"T = {T}K, L = {L_1 * 1e12}pH",
                                                                                   params=params, cutlineGeneratingFunction = False, 
                                                                                   cutlineInformation = cutlineInformation, particleInformation = particleInformation, contourData = contourData,
                                                                                   pColor=pColor, contour_range = [0, 500], cutlineXLimit=[-4, 4], cutlineYLimit=[0, 500])
    fig.tight_layout()
    writer = animation.PillowWriter(fps=10, metadata=dict(artist='Me'), bitrate=1800)
    ani.save('scatter.gif', writer=writer)
    plt.close()
    


In [ ]:
Image(filename="scatter.gif")

### with cutlineGeneratingFunction

In [ ]:
animation_play = True
# animation_play = False
if animation_play:

    frame_skip = int(1/50 * 1/params['dt'])
    print("geenerated at ", datetime.datetime.now())
    targetTimeArray = protocol_all_time_array[simResult['cfqr'].sim.target_step_index]
    ani, fig, ax = coupled_fq_protocol_library.animate_sim_flux_qubit_with_cutline_and_projection(simResult['cfqr'], time_array=targetTimeArray, frame_skip = frame_skip, plot_axis= [0,1],
                                                                                   vmin=graph_setting[T]['vmin'], vmax=graph_setting[T]['vmax'], ax0_title = f"T = {T}K, L = {L_1 * 1e12}pH",
                                                                                   params=params, cutlineGeneratingFunction = cutlineDataGeneratingFunction(initial_parameter_dict),
                                                                                   pColor=pColor, contour_range = [0, 500], cutlineXLimit=[-4, 4], cutlineYLimit=[0, 500])
    fig.tight_layout()
    writer = animation.PillowWriter(fps=15, metadata=dict(artist='Me'), bitrate=1800)
    ani.save('scatter.gif', writer=writer)
    Image(filename="scatter.gif")

### save data analysis

In [ ]:
print(f"current comment: {params['comment']}")
response = input("Are you happy with the current comment? If not, give me a new comment.")
if response == "y":
    pass
else:
    params['comment'] = response

In [ ]:
simResult['error_rate_array']

In [ ]:
importlib.reload(sys.modules['edward_tools.cfq_batch_sweep'])

In [ ]:
cfq_batch_sweep.saveSimulationResult(simResult, U0_1, timeOrStep = 'step', save = True, save_final_state = False, saveFolderPath = "coupled_flux_qubit_protocol/coupled_flux_qubit_data_gallery", save_all_state = False)

# save state

In [ ]:
target_state = all_states[:,106, ...]
plt.scatter(target_state[..., 0, 0], target_state[..., 1, 0])

In [ ]:
state_name = "just_before_KEP_N_1000.npy"
np.save("saved_init_state/" + state_name, target_state)

# save pickle

In [ ]:
experiment_label = "Experiment 8 (2025-1-27)" 

In [ ]:
with open(f'coupled_flux_qubit_protocol/simulation_protocol_history/{experiment_label}.pickle', "rb") as f:
    k = pickle.load(f)

In [ ]:
k['params']['comment']

In [ ]:
export_cfqr_params = params['circuit_parameters']

export_params = params
export_params['comment'] = f"{experiment_label}: To study the trade-off between speed and work done, and also speed and fidelity CE_t_factor = 10"


export_object = {
    "cfqr_params": export_cfqr_params,
    "params": export_params,
    "initial_external_parameter_dict": initial_parameter_dict,
    "protocol_list": protocol_list,
    "manual_domain": [np.array([-5, -5]), np.array([5, 5])]
}
    

print(export_params['comment'])
print(simResult['simulation_data']['simulation_id'])

In [ ]:
pickle_file_path = os.path.join("coupled_flux_qubit_protocol", "simulation_protocol_history", f"{experiment_label}.pickle")

with open(pickle_file_path, "wb") as f:
    pickle.dump(export_object, f)

In [ ]:
# modify pickle

In [ ]:
modify_item = [(0, 200), (40, 160), (80, 120), (120, 80), (160, 40), (200, 0)]

In [ ]:
import copy


In [ ]:
k_array = []

In [ ]:
for _i, (t_1, t_2) in enumerate(modify_item):
    k_template = copy.deepcopy(k)

        
    k_template['protocol_list'][1]['duration'] = t_1
    k_template['protocol_list'][2]['duration'] = t_2
    
    if t_1 == 0:
        k_template['protocol_list'].pop(1)
    if t_2 == 0:
        k_template['protocol_list'].pop(2)
    k_array.append(k_template)
    
    pickle_file_path = os.path.join("coupled_flux_qubit_protocol", "simulation_protocol_history", f"{experiment_label}_series_{_i}.pickle")

    with open(pickle_file_path, "wb") as f:
        pickle.dump(k_template, f)
    

In [ ]:
for x in k_array:
    print(x['protocol_list'])

In [ ]:
pickle_prefix = "Experiment 1 (2024-11-12)_series_"
# pickle_prefix = "Experiment 1 (2024-10-27)"
print(pickle_prefix)
comment = f"Experiment 1 (2024-11-12) CE operation, with R = 300Ω and various ratio of the KE protocol"
param_dt = 1/100
loopTime = 100
protocol_type = "CE"






####################
# no need to change the following 
####################

pickle_file_base_folder = "coupled_flux_qubit_protocol/simulation_protocol_history"

all_pickle_file = os.listdir(pickle_file_base_folder)
target_pickle_file_list = [os.path.join(pickle_file_base_folder, p) for p in all_pickle_file if pickle_prefix in p]


In [ ]:
target_pickle_file_list

In [ ]:
particle_index = 0
particle_category = "00"
time_range = None

index_choice = simResult['cfqr'].getIndexOfParticles()["00"]
all_states = simResult['cfqr'].sim.output.all_state['states']
all_states_00_i1 = all_states[index_choice][particle_index, ...]

In [ ]:
_, _, protocol_time_array, protocol_time_index_array = simResult['cfqr'].createProtocolTimeArray(protocol_list, params)

protocol_all_time_index_array = np.array(range(0, int(np.array(protocol_time_array[-1]) / params['dt']) + 1))
protocol_all_time_array  = protocol_all_time_index_array * params['dt']



In [ ]:
reduced_protocal_time_array = simResult['cfqr'].getKeyTime()
KE_of_the_particle = simResult['cfqr'].system.get_kinetic_energy(all_states_00_i1)
PE_of_the_particle = [simResult['cfqr'].system.get_potential(_s, _t ) for _t, _s in  zip(reduced_protocal_time_array, all_states_00_i1)]
WD_of_the_particle = simResult['cfqr'].sim.work_dist_time_array_whole_process[index_choice][particle_index]

In [ ]:
keyState_choosen_particle = [ s[index_choice][particle_index, ...] for s in simResult['cfqr'].sim.keyStep_all_states]
total_energy = PE_of_the_particle - min(PE_of_the_particle) + KE_of_the_particle
adjusted_PE = PE_of_the_particle - PE_of_the_particle[0]
adjusted_total_energy = adjusted_PE + KE_of_the_particle
characteristic_t = np.sqrt(params['circuit_parameters']['L_1'] * params['circuit_parameters']['L_factor'] * params['circuit_parameters']['C_1'] * params['circuit_parameters']['C_factor'])


In [ ]:
reduced_protocal_time_array

In [ ]:
def search_protocol_time_array_index(array, start, end):
    x = np.sort(np.append(array, start))
    y = np.sort(np.append(array, end))
    
    start_index = np.where(x == start)[0][0]
    if start != round(array[start_index]):
        start_index += 1
    
    
    final_index = np.where(y == end)[0][0] 
    if end != round(array[final_index]):
        final_index -= 1

    return slice(start_index, final_index)

In [ ]:
index_slice = search_protocol_time_array_index(reduced_protocal_time_array, 23, 60.)

# reduced_protocal_time_array[index_slice]
# search_protocol_time_array_range(adjusted_PE, 23, 60.)

In [ ]:
reduced_protocal_time_array[index_slice]
KE_of_the_particle[None]

In [ ]:
export_object

In [ ]:
def single_particle_analysis(simResult, params, protocol_list, plot_ax = None, particle_index = 0, particle_category = "00", time_range = None):
    

    
    
    # _ = [simResult['cfqr'].system.get_potential(_s, _t ) for _s, _t in zip(keyState_choosen_particle, simResult['cfqr'].protocol_time_array)]
    
    
    

    if plot_ax == None:
        plt.figure(figsize=(8, 5))
    plot_ax.plot(protocal_key_time_array, adjusted_PE , label = "PE", color = "#CC79A7")
    plot_ax.plot(protocal_key_time_array, KE_of_the_particle, label = "KE")
    plot_ax.plot(protocal_key_time_array, adjusted_total_energy, label = "KE + PE")
    plot_ax.plot(protocal_key_time_array, WD_of_the_particle[::100], label = "WD", color = "#000000")
    plot_ax.plot(protocal_key_time_array, WD_of_the_particle[::100] - adjusted_total_energy, label = "energy loss")
    plot_ax.set_ylabel(r"energy $(k_BT)$")
    plot_ax.set_xlabel(f"time ({characteristic_t * 1e9:.3g} ns)")
    plot_ax.legend(bbox_to_anchor=(1.25, 1.05))

In [ ]:
os.path.join("~", "source", "coupled_flux_qubit_protocol", "simulation_protocol_history", f"{245}.pickle")




In [ ]:
"/Users/tkwtang/source/coupled_flux_qubit_protocol/simulation_protocol_history/"

# circuit parameter at different t and ∆U

In [ ]:
reduced_time_array = cfqr.getReducedTimeArray()

In [ ]:
params_at_t = [cfqr.convert_params_list_into_dict(cfqr.protocol.get_params(_t)) for _t in reduced_time_array]

In [ ]:
circuit_parameters_at_different_t = []

for x in params_at_t:
    circuit_parameter_template = experiment_circuit_params.copy()
    circuit_parameter_template['phi_1x'] = x['phi_1_x']
    circuit_parameter_template['phi_2x'] = x['phi_2_x']
    circuit_parameter_template['phi_1xdc'] = x['phi_1_dcx']
    circuit_parameter_template['phi_2xdc'] = x['phi_2_dcx']
    circuit_parameter_template['M_12'] = x['M_12']
    circuit_parameters_at_different_t.append(circuit_parameter_template)

In [ ]:
from edward_tools.pot_analysis_helper_functions import get_XYU, pot_function, find_all_critical_points_for_all_potential

In [ ]:
contour_data_array = [get_XYU(circuit_params, domain=[-4, 4], resolution=500) for circuit_params in circuit_parameters_at_different_t]


In [ ]:
initial_guess = [(-2, -2), (-2, 2), (2, -2), (2, 2), (-2,0), (0, -2), (2, 0), (0, 2)]
critical_points_color = ["green", "red", "yellow", "orange", "white", "white", "white", "white"]

In [ ]:
k_BT

In [ ]:
result_array = []
for _cp in circuit_parameters_at_different_t:
    critical_dict = find_all_critical_points_for_all_potential(_cp,
                    guess = [(-2, -2), (-2, 2), (2, -2), (2, 2), \
                             (-2,0), (0, -2), (2, 0), (0, 2)])
    critical_points = list(critical_dict.values())[0]
    critical_potential = [pot_function(_cp)([x, y]) for x, y in critical_points]
    delta_U00 = critical_potential[4] - critical_potential[1]
    delta_U01 = critical_potential[5] - critical_potential[0]
    delta_U10 = critical_potential[6] - critical_potential[2]
    delta_U11 = critical_potential[6] - critical_potential[3]

    X, Y, U = get_XYU(_cp, [-4, 4], 50)

    result_array.append({
        "coord": critical_points, "delta_U00": delta_U00, "delta_U01": delta_U01,
        "delta_U10": delta_U10, "delta_U11": delta_U11
    })


In [ ]:
delta_U_data = [(item['delta_U00'], item['delta_U01'], item['delta_U10'], item['delta_U11'])  for item in result_array]

In [ ]:
total_duration = protocol_time_array[-1] * characteristic_time * 1e9

plt.plot(reduced_time_array * characteristic_time * 1e9, U0_1 * np.array(delta_U_data)[:, 0], label = "∆U00")
plt.plot(reduced_time_array * characteristic_time * 1e9, U0_1 * np.array(delta_U_data)[:, 1], label = "∆U01")
plt.plot(reduced_time_array * characteristic_time * 1e9, U0_1 * np.array(delta_U_data)[:, 2], label = "∆U10")
plt.plot(reduced_time_array * characteristic_time * 1e9, U0_1 * np.array(delta_U_data)[:, 3], label = "∆U11")
xmin, xmax = plt.xlim()
plt.hlines(y = 50, xmin = xmin, xmax = xmax, linestyle = "--")
plt.legend()
plt.xlabel(r'time $(ns)$')
plt.ylabel(r'energy $(k_BT)$')
plt.title(f"∆U00 vs t (total duration = {total_duration:.3g}ns) ")


# save all states

In [ ]:
base_folder = "cfq_data/sudden_jump_KEP/"
prefix = "KEP(jump)"
name_of_state_file = prefix + "_all_state.npy"
name_of_reduced_time_array_file = prefix + "_reduced_time_array.npy"
name_of_index_file = prefix + "_index_of_particle.npy"

np.save(os.path.join(base_folder, name_of_reduced_time_array_file), reduced_protocal_time_array)
np.save(os.path.join(base_folder, name_of_state_file), all_states)
np.save(os.path.join(base_folder, name_of_index_file), simResult['cfqr'].getIndexOfParticles())